In [155]:
import json
import random
import re
import requests
import pandas as pd
import csv 
import time


In [4]:
# url and user to use the API rest

endponint_get = 'https://7b8uflffq0.execute-api.us-east-1.amazonaws.com/game/get_params'

endponint_post = 'https://7b8uflffq0.execute-api.us-east-1.amazonaws.com/game/check_results'

user = ('juan.bedoya','6b2a4f8d28204cf0a3286d1f9213ecfc')

In [5]:
# open data base (clean words)

with open('Data_Base_Final_Words.txt', 'r',encoding= 'utf-8') as data_file:

  logs_text = data_file.read()

words_final_list = logs_text.split()
print(len(words_final_list))

32430


In [246]:
# create dataframe from words list to know the frecuency of letters positions

def create_dataframe_positions(words_list):
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
                'n','ñ', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
        
    position_letters_dict = posittion_dict(alphabet,words_list)

    letter_list = [i for i in position_letters_dict]
    df_words_frecuency= pd.DataFrame(list(zip(letter_list)),
                        columns =['letters'])

    for i in range(len(words_list[0])):

        df_words_frecuency['position_{}'.format(i)] = [position_letters_dict[j].count(i)/len(words_list) for j in position_letters_dict]

    return df_words_frecuency.set_index('letters')

# select word from list base on the dataframe

def choose_word(words_list):

    df_words_frecuency = create_dataframe_positions(words_list)

    chosee_weight = 0
    for word in words_list:
        weight = 0
        for position,letter in enumerate(word):
            weight += df_words_frecuency.loc[letter,'position_{}'.format(position)] / word[position:].count(letter)
        if weight > chosee_weight:
            select_word = word
            chosee_weight = weight
    
    return select_word

# create dict of letters and their respective position on a list of words

def posittion_dict(letters_list,words_list):
    dicts = {}
    for letter in letters_list:
        for word in words_list:
            for letter1 in range(len(word)):
                if letter == word[letter1]:
                    if letter not in dicts:
                        dicts[letter] = []
                    dicts[letter].append(letter1)
                    
    return dicts

# filter words by lenght

def words_specific_length(words_list,long):    
    words = []
    for word in words_list:
        if len(word) == long:
            words.append(word)
    return words

# filter words by number of vowels

def words_by_vowels(words_list,num_vowels):
    word_list_by_vowels = []
    for word in words_list:
        count_vowels = len(re.findall("[aeiou]", word))
        if count_vowels == num_vowels:
            word_list_by_vowels.append(word)
    return word_list_by_vowels

# add to the dic golbal variable new element 

def dic_wrong_positions(positions_list,wrong_positions_letters,last_word):
    global dic_wrong_positions_letters
    for position, letter in enumerate(last_word):
        if letter in wrong_positions_letters:
            if positions_list[position]== False:
                if letter not in dic_wrong_positions_letters:
                    dic_wrong_positions_letters[letter] = []
                dic_wrong_positions_letters[letter].append(position)
        if positions_list[position] == True and letter not in dic_wrong_positions_letters:
            dic_wrong_positions_letters[letter] = []
    
    return dic_wrong_positions_letters

# create letters list that aren't on the word

def wrong_letters(positions_list,dict_letters_wposition,last_word):
    global list_wrong_letters
    for position, letter in enumerate(last_word):
        if letter not in dict_letters_wposition and positions_list[position] != True:
            list_wrong_letters.append(letter)
    
    return list_wrong_letters

# filter words list that have the letter on the right position

def correct_position_words(position_list,last_word,words_list):
    possible_words = []
    for word in words_list:
        count = 0
        for position, letter in enumerate(word):
            if letter == last_word[position] and position_list[position] == True:
                count += 1
        if count == sum(position_list):
            possible_words.append(word)
    return possible_words

# remove word that have letter in a list 

def words_without_wrong_letters(words_list,letters_list):
    possible_words = words_list[:]
    for word in words_list:
        for letter in letters_list:
            if letter in word:
                possible_words.remove(word)
                break
    return possible_words

# remove word taht have a letter in a wrong position

def words_without_letters_wposition(words_list,dict_letters_wposition):
    possible_words = words_list[:]
    for word in words_list:
        for position ,letter in enumerate(word):
            if letter in dict_letters_wposition:
                if position in dict_letters_wposition[letter]:
                    possible_words.remove(word)
                    break
    return(possible_words)

# remove words if don't have a letter that is in the search word 

def words_with_letters_in(words_list,dict_letters_wpossition):
    list_letters_in = list(dict_letters_wpossition)
    possible_words = words_list[:]
    for word in words_list:
        for letter in list_letters_in:
            find = re.findall(letter,word)
            if find == []:
                possible_words.remove(word)
                break
    return possible_words
        
# function to find the next posible word using the last function in order

def find_next_word(words_list,last_word,position_list,wrong_positions_letter):

    create_dic_wrong_positions = dic_wrong_positions(position_list,wrong_positions_letter,last_word)
    
    list_wrong_letters = wrong_letters(position_list,create_dic_wrong_positions,last_word)

    words_with_correct_letters = correct_position_words(position_list,last_word,words_list)

    words_without_wletters = words_without_wrong_letters(words_with_correct_letters,list_wrong_letters)

    words_without_wposition = words_without_letters_wposition(words_without_wletters,create_dic_wrong_positions)

    words_with_wpossition_letters = words_with_letters_in(words_without_wposition,create_dic_wrong_positions)

    return words_with_wpossition_letters





In [164]:
#Create .csv file to save games data

'''header = ["word", "word_lenght","vowels","consonants","attempts","date","total_exec_time","game_exec_time"]

with open("games_data.csv", "w", encoding="UTF8") as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
'''

In [249]:
#play the game from API REST
total_start = time.time()
dic_wrong_positions_letters = {}
dic_correct_positions_letters = {}
list_wrong_letters = []
limit = 6
response_get_letter = requests.get(endponint_get, auth = user)
json = response_get_letter.json()

game_start = time.time()
word_data = [json["length_word"],json["vowels"],json["consonants"]]
words_by_lenght = words_specific_length(words_final_list,json["length_word"])

words_to_play = words_by_vowels(words_by_lenght,json["vowels"])

select_word = choose_word(words_to_play)

for i in range(limit):
    json_response ={
    "result_word" : select_word
    }
    response_post_word = requests.post(endponint_post,json = json_response,auth = user)
    json = response_post_word.json()
    if json["score"] >= 1:
        break
    possible_words = find_next_word(words_to_play,json["word_sent"],json["position_array"],json["right_letters_in_wrong_positions"])
    words_to_play = possible_words
    select_word = choose_word(possible_words)

end_game = time.time()
total_time = (end_game - total_start)
game_time = (end_game - game_start)

record_data = [json["word_sent"],word_data[0],word_data[1],word_data[2],json["current_attemps"],
               json["try_datetime"],total_time,game_time]
               
with open("games_data.csv", "a", encoding="UTF8") as f:
    writer = csv.writer(f)

    # write data
    writer.writerow(record_data)





In [250]:
print(json)

{'word_sent': 'posicionamiento', 'score': 1.0, 'try_datetime': '2022-06-30T03:51:12.751819', 'position_array': [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True], 'right_letters_in_wrong_positions': [], 'current_attemps': 2}


In [211]:
# local test choose random word 

dic_wrong_positions_letters = {}
dic_correct_positions_letters = {}
list_wrong_letters = []

json = {
  "id": "string",
  "length_word": 13,
  "vowels": 5,
  "consonants": 3
}

words_list_for_game = words_specific_length(words_final_list,json["length_word"])
#print(dummy)
words_list_for_game = words_by_vowels(words_list_for_game,json["vowels"])
#print(filt)

random_word = random.choice(words_list_for_game)

#random_word1 = random.choice(words_list_for_game)
random_word1 = choose_word(words_list_for_game)
cont = 1
print(random_word)
print(random_word1)

while random_word1 != random_word or cont == 5:
  array = []
  for pos,let in enumerate(random_word1):
    if let == random_word[pos]:
      array.append(True)
    else:
      array.append(False)
  letters = []
  for position, letter in enumerate(random_word1):
    if letter in random_word and array[position] == False:
      letters.append(letter)
  # print(array)
  # print(letters)
  json = {
    "word_sent": random_word1,
    "score": 0.233,
    "try_datetime": '21-06-2022',
    "position_array": array,
    "right_letters_in_wrong_positions": letters
  }

  cont += 1
  print(cont)

  find_words = find_next_word(words_list_for_game,json["word_sent"],json["position_array"],json["right_letters_in_wrong_positions"])
  #print(find)
  random_word1 = choose_word(find_words)
  print(random_word)
  print(random_word1)



tlamatiliztli
centralizadas
2
tlamatiliztli
tlamatiliztli


In [238]:
dic_wrong_positions_letters = {}
dic_correct_positions_letters = {}
list_wrong_letters = []
random_word1 = 'centralizadas'#prehistoricas
words_list_for_game = words_specific_length(words_final_list,13)
#print(dummy)
words_list_for_game = words_by_vowels(words_list_for_game,5)
json = {
"word_sent": random_word1,
"score": 0.233,
"try_datetime": '21-06-2022',
"position_array": [False, False, False, False, False, False, False, False, False, False, False, True, True],
"right_letters_in_wrong_positions": ['c', 'e', 't', 'r', 'i']
}

cont += 1
print(cont)

find_words = find_next_word(words_list_for_game,json["word_sent"],json["position_array"],json["right_letters_in_wrong_positions"])
print(find_words)
print(list_wrong_letters)
print(dic_wrong_positions_letters)

19
{'c': [0], 'e': [1], 't': [3], 'r': [4], 'i': [7], 'a': [], 's': []}
['prehistoricas']
['n', 'l', 'z', 'd']
{'c': [0], 'e': [1], 't': [3], 'r': [4], 'i': [7], 'a': [], 's': []}
